In [14]:
import pandas as pd
from google.colab import drive
import os
from zipfile import ZipFile
import numpy as np
import json


drive.mount('/content/drive')

testgenes = pd.read_csv("/content/drive/MyDrive/Fitness_Map_Group_Project/top_interaction_sequences.csv")
lowtestgenes = pd.read_csv("/content/drive/MyDrive/Fitness_Map_Group_Project/least_interaction_sequences.csv")

testgenes = pd.concat([testgenes,lowtestgenes])
testgenes.reset_index(inplace = True)
testgenes.drop('Unnamed: 0',axis = 1,inplace = True)


colabpath = r"/content"

AF_outputs = r"/content/drive/MyDrive/Fitness_Map_Group_Project/Alphafold_2_outputs"   #Replace with the path to the Alphafold_2_outputs folder for your mounted drive. Our example folder is here: https://drive.google.com/drive/folders/1Gz7YlvsNBLRM48aVSyd6X7ODeR3TPklX?usp=drive_link
AF_server_outputs = r"/content/drive/MyDrive/Fitness_Map_Group_Project/Alphafold_2_outputs/Predictions_From_AlphaFold_Server_AlphaFold3"

testgenes["Min_PAE_of_Interaction"] = float("nan")
testgenes["Mean_PAE_of_Interaction"] = float("nan")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
def get_pae(chainA_len, chainB_len):

  chain_A_range = range(0, chainA_len)    #residue indexes for chain A
  chain_B_range = range(chainA_len, chainA_len + chainB_len)   # residue indices for chain B


  # Extract the inter-chain PAE block
  inter_chain_pae = pae[np.ix_(chain_A_range, chain_B_range)]

  # Compute metrics
  min_pae = np.min(inter_chain_pae)
  mean_pae = np.mean(inter_chain_pae)     #currently, mean PAE is only calculated for AlphaFold 2 outputs. However, this can be extended to the AlphaFold 3 outputs as well.

  print(f"Minimum PAE between chains: {min_pae:.2f}")
  print(f"Mean PAE between chains: {mean_pae:.2f}")

  return min_pae, mean_pae

In [16]:
for folder in os.listdir(AF_outputs):
  if ".zip" in folder:
    with ZipFile((os.path.join(AF_outputs,folder)), 'r') as zObject:

    # Extracting all the members of the zip into google colab
      zObject.extractall(
          path="/content")
    gene_pair = folder[:folder.find(".result.zip")]
    first_ = gene_pair.find("_")
    second_ = gene_pair.find("_",first_+1)
    third_ = gene_pair.find("_",second_+1)
    gene1 = int(gene_pair[first_+1:second_])
    gene2 = int(gene_pair[second_+1:third_])
    print(gene1,gene2)


    pae_file = gene_pair + "_predicted_aligned_error_v1.json"
    with open("/content/" + os.path.join(gene_pair, pae_file)) as f:
      pae_data = json.load(f)
    pae = np.array(pae_data['predicted_aligned_error'])

    #Note:
    # pae.shape
    # len(testgenes.loc[0,"gene1_seq"]) + len(testgenes.loc[0,"gene2_seq"])  #the shape of the pae array is the length of the sequences added together

    gene_row = testgenes[(testgenes["gene1_locusId"] == gene1) & (testgenes["gene2_locusId"] == gene2)]
    gene1_seq = gene_row["gene1_seq"].item()
    gene2_seq = gene_row["gene2_seq"].item()

    print(len(gene1_seq))
    print(len(gene2_seq))

    min_pae, mean_pae = get_pae(len(gene1_seq), len(gene2_seq))

    testgenes.loc[gene_row.index,"Min_PAE_of_Interaction"] = min_pae

    testgenes.loc[gene_row.index,"Mean_PAE_of_Interaction"] = mean_pae



17245 15651
541
144
Minimum PAE between chains: 18.81
Mean PAE between chains: 25.95
17552 17261
350
84
Minimum PAE between chains: 7.26
Mean PAE between chains: 16.28
18059 17626
491
440
Minimum PAE between chains: 25.34
Mean PAE between chains: 29.34
17552 18130
350
241
Minimum PAE between chains: 14.21
Mean PAE between chains: 24.05
17552 16347
350
452
Minimum PAE between chains: 23.25
Mean PAE between chains: 28.82
17552 15217
350
413
Minimum PAE between chains: 15.39
Mean PAE between chains: 26.20
17552 17788
350
346
Minimum PAE between chains: 10.05
Mean PAE between chains: 22.38
14285 17261
865
84
Minimum PAE between chains: 14.19
Mean PAE between chains: 22.70
18059 17625
491
484
Minimum PAE between chains: 22.20
Mean PAE between chains: 28.55
17937 15399
582
102
Minimum PAE between chains: 21.34
Mean PAE between chains: 29.39
18059 17630
491
392
Minimum PAE between chains: 18.77
Mean PAE between chains: 27.39
17552 18132
350
122
Minimum PAE between chains: 12.64
Mean PAE betwe

In [17]:
#This function is to obtain min PAE values from the outputs obtained on the AlphaFold server (https://alphafoldserver.com/)
#Not that this does not calculate mean PAEs-- if you want to add this calculation, you'll need to add code to use the PAE matrix file instead of the summary_confidences file, in the same way that the get_pae() function analyzed the AlphaFold 2 PAE matrix


def get_AF_server_minPAE(parent_folder_path):
  min_paes = []
  for item in os.listdir(parent_folder_path):
      if "summary_confidences_" in item:
        print(item)
        output_path = os.path.join(parent_folder_path,item)
        f = open(output_path)
        data = json.load(f)
        pair_pae = data["chain_pair_pae_min"]
        min_paes.append(pair_pae[0][1])
        min_paes.append(pair_pae[1][0])
  if len(min_paes) != 10:
    print(f"wrong number of summary_confidences files:{len(min_paes)/2}")
  to_add = min(min_paes)
  return(to_add)



In [18]:
AF_server_outputs = r"/content/drive/MyDrive/Fitness_Map_Group_Project/Alphafold_2_outputs/Predictions_From_AlphaFold_Server_AlphaFold3"

for folder in os.listdir(AF_server_outputs):
  os.makedirs("/content/" + folder[:-4], exist_ok=True)
  if ".zip" in folder:
    with ZipFile((os.path.join(AF_server_outputs,folder)), 'r') as zObject:

    # Extracting all the members of the zip into google colab
      zObject.extractall(
          path="/content/"+folder[:-4])

    gene_pair = folder[folder.find("_")+1:folder.find("zip")]   #folders are titled in the style 'fold_locusids_1937043_12785254.zip'
    first_ = gene_pair.find("_")
    second_ = gene_pair.find("_",first_+1)
    third_ = gene_pair.find("_",second_+1)
    gene1 = int(gene_pair[first_+1:second_])
    gene2 = int(gene_pair[second_+1:third_])
    print(gene1,gene2)


    gene_row = testgenes[(testgenes["gene1_locusId"] == gene1) & (testgenes["gene2_locusId"] == gene2)]
    gene1_seq = gene_row["gene1_seq"].item()
    gene2_seq = gene_row["gene2_seq"].item()

    print(len(gene1_seq))
    print(len(gene2_seq))

    min_pae = get_AF_server_minPAE("/content/" + folder[:-4])

    testgenes.loc[gene_row.index,"Min_PAE_of_Interaction"] = min_pae
    testgenes.loc[gene_row.index,"Mean_PAE_of_Interaction"] = "Add later"
    print(testgenes.loc[gene_row.index])


14285 18129
865
150
fold_locusids_14285_18129_summary_confidences_4.json
fold_locusids_14285_18129_summary_confidences_0.json
fold_locusids_14285_18129_summary_confidences_3.json
fold_locusids_14285_18129_summary_confidences_2.json
fold_locusids_14285_18129_summary_confidences_1.json
    index     gene1  gene1_locusId  \
16     16  Gene_103          14285   

                                            gene1_seq      gene2  \
16  MTIEYTKNYHHLTRIATFCALLYCNTAFSAELVEYDHTFLMGQNAS...  Gene_3236   

    gene2_locusId                                          gene2_seq  \
16          18129  MHADTATRQHWMSVLAHSQPAELAARLNALNITADYEVIRAAETGL...   

    coefficient  Min_PAE_of_Interaction Mean_PAE_of_Interaction  
16     3.422726                   24.92               Add later  


<ipython-input-18-2322890363>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Add later' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  testgenes.loc[gene_row.index,"Mean_PAE_of_Interaction"] = "Add later"


16738 18132
1526
122
fold_locusids_16738_18132_summary_confidences_3.json
fold_locusids_16738_18132_summary_confidences_4.json
fold_locusids_16738_18132_summary_confidences_0.json
fold_locusids_16738_18132_summary_confidences_2.json
fold_locusids_16738_18132_summary_confidences_1.json
    index      gene1  gene1_locusId  \
10     10  Gene_2151          16738   

                                            gene1_seq      gene2  \
10  MNRTSPYYCRRSVLSLLISALIYAPPGMAAFTTNVIGVVNDETVDG...  Gene_3238   

    gene2_locusId                                          gene2_seq  \
10          18132  MLALFIHTTGVLSKLLSEAVEAIEPGPVEGIRATGANKLEEILYGV...   

    coefficient  Min_PAE_of_Interaction Mean_PAE_of_Interaction  
10      3.61889                   25.91               Add later  
1937043 12785254
1157
17
fold_locusids_1937043_12785254_summary_confidences_3.json
fold_locusids_1937043_12785254_summary_confidences_4.json
fold_locusids_1937043_12785254_summary_confidences_0.json
fold_locusids_1937043_

In [19]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

testgenes.dropna(inplace = True)
testgenes.reset_index(drop=True, inplace = True)
testgenes

,index,gene1,gene1_locusId,gene1_seq,gene2,gene2_locusId,gene2_seq,coefficient,Min_PAE_of_Interaction,Mean_PAE_of_Interaction
0,0,Gene_2567,17245,MTVFNKFARTFKSHWLLYLCVIVFGITNLVASSGAHMVQRLLFFVL...,Gene_1225,15651,MKSTSDLFNEIIPLGRLIHMVNQKKDRLLNEYLSPLDITAAQFKVL...,5.033844e+00,18.81,25.954227
1,1,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_2574,17261,MENNEIQSVLMNALSLQEVHVSGDGSHFQVIAVGELFDGMSRVKKQ...,4.562073e+00,7.26,16.279268
2,2,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_1817,16347,MLSIFKPAPHKARLPAAEIDPTYRRLRWQIFLGIFFGYAAYYLVRK...,4.355901e+00,23.25,28.821514
3,3,Gene_3174,18059,MNTQYNSSYIFSITLVATLGGLLFGYDTAVISGTVESLNTVFVAPQ...,Gene_2850,17626,MQAYFDQLDRVRYEGSKSSNPLAFRHYNPDELVLGKRMEEHLRFAA...,4.136822e+00,25.34,29.340512
4,4,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_3237,18130,MHLSTHPTSYPTRYQEIAAKLEQELRQHYRCGDYLPAEQQLAARFE...,4.050859e+00,14.21,24.04674
5,5,Gene_3174,18059,MNTQYNSSYIFSITLVATLGGLLFGYDTAVISGTVESLNTVFVAPQ...,Gene_2849,17625,MYIGIDLGTSGVKVILLNEQGEVVAAQTEKLTVSRPHPLWSEQDPE...,3.873120e+00,22.20,28.553247
6,6,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_879,15217,MSKRRVVVTGLGMLSPVGNTVESTWKALLAGQSGISLIDHFDTSAY...,3.862169e+00,15.39,26.197694
7,7,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_2982,17788,MKVMRTTVATVVAATLSMSAFSVFAEASLTGAGATFPAPVYAKWAD...,3.846149e+00,10.05,22.378076
8,10,Gene_2151,16738,MNRTSPYYCRRSVLSLLISALIYAPPGMAAFTTNVIGVVNDETVDG...,Gene_3238,18132,MLALFIHTTGVLSKLLSEAVEAIEPGPVEGIRATGANKLEEILYGV...,3.618890e+00,25.91,Add later
9,11,Gene_3076,17937,MLNERQLKIVDLLEQQPRTPGELAQQTGVSGRTILRDIDYLNFTLN...,Gene_1019,15399,MKYLLIFLLVLAIFVISVTLGAQNDQQVTFNYLLAQGEYRISTLLA...,-3.501067e+00,21.34,29.391092


In [20]:
testgenes

,index,gene1,gene1_locusId,gene1_seq,gene2,gene2_locusId,gene2_seq,coefficient,Min_PAE_of_Interaction,Mean_PAE_of_Interaction
0,0,Gene_2567,17245,MTVFNKFARTFKSHWLLYLCVIVFGITNLVASSGAHMVQRLLFFVL...,Gene_1225,15651,MKSTSDLFNEIIPLGRLIHMVNQKKDRLLNEYLSPLDITAAQFKVL...,5.033844e+00,18.81,25.954227
1,1,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_2574,17261,MENNEIQSVLMNALSLQEVHVSGDGSHFQVIAVGELFDGMSRVKKQ...,4.562073e+00,7.26,16.279268
2,2,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_1817,16347,MLSIFKPAPHKARLPAAEIDPTYRRLRWQIFLGIFFGYAAYYLVRK...,4.355901e+00,23.25,28.821514
3,3,Gene_3174,18059,MNTQYNSSYIFSITLVATLGGLLFGYDTAVISGTVESLNTVFVAPQ...,Gene_2850,17626,MQAYFDQLDRVRYEGSKSSNPLAFRHYNPDELVLGKRMEEHLRFAA...,4.136822e+00,25.34,29.340512
4,4,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_3237,18130,MHLSTHPTSYPTRYQEIAAKLEQELRQHYRCGDYLPAEQQLAARFE...,4.050859e+00,14.21,24.04674
5,5,Gene_3174,18059,MNTQYNSSYIFSITLVATLGGLLFGYDTAVISGTVESLNTVFVAPQ...,Gene_2849,17625,MYIGIDLGTSGVKVILLNEQGEVVAAQTEKLTVSRPHPLWSEQDPE...,3.873120e+00,22.20,28.553247
6,6,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_879,15217,MSKRRVVVTGLGMLSPVGNTVESTWKALLAGQSGISLIDHFDTSAY...,3.862169e+00,15.39,26.197694
7,7,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_2982,17788,MKVMRTTVATVVAATLSMSAFSVFAEASLTGAGATFPAPVYAKWAD...,3.846149e+00,10.05,22.378076
8,10,Gene_2151,16738,MNRTSPYYCRRSVLSLLISALIYAPPGMAAFTTNVIGVVNDETVDG...,Gene_3238,18132,MLALFIHTTGVLSKLLSEAVEAIEPGPVEGIRATGANKLEEILYGV...,3.618890e+00,25.91,Add later
9,11,Gene_3076,17937,MLNERQLKIVDLLEQQPRTPGELAQQTGVSGRTILRDIDYLNFTLN...,Gene_1019,15399,MKYLLIFLLVLAIFVISVTLGAQNDQQVTFNYLLAQGEYRISTLLA...,-3.501067e+00,21.34,29.391092


In [21]:
top_hits = testgenes.copy()
low_hits = testgenes.loc[27:28,:]

top_hits.drop(27, inplace = True)
top_hits.drop(28, inplace = True)
top_hits_display = top_hits[["gene1_locusId","gene2_locusId","coefficient","Min_PAE_of_Interaction"]].sort_values(by=["Min_PAE_of_Interaction"])

low_hits_display = low_hits[["gene1_locusId","gene2_locusId","coefficient","Min_PAE_of_Interaction"]].sort_values(by=["Min_PAE_of_Interaction"])



top_hits.describe()

,index,gene1_locusId,gene2_locusId,coefficient,Min_PAE_of_Interaction
count,27.000000,2.700000e+01,2.700000e+01,27.000000,27.000000
mean,15.962963,1.590364e+05,1.176189e+06,3.313039,17.586296
std,10.878636,5.123664e+05,3.401572e+06,1.450659,6.110726
min,0.000000,1.428500e+04,1.521700e+04,-3.501067,6.000000
25%,6.500000,1.594250e+04,1.639500e+04,3.193200,13.415000
50%,15.000000,1.755200e+04,1.762600e+04,3.422726,18.810000
75%,25.000000,1.792550e+04,1.813100e+04,3.854159,22.405000
max,41.000000,1.937043e+06,1.278525e+07,5.033844,25.910000


In [22]:
low_hits_display

,gene1_locusId,gene2_locusId,coefficient,Min_PAE_of_Interaction
28,15045,14698,4.703083e-09,17.53
27,17437,14332,4.520093e-09,25.03


In [23]:
testgenes.to_csv("top_and_low_interaction_PAE.csv")